In [2]:
import sidpy
import h5py

In [3]:
h5_f = h5py.File('./Data/data_raw_unmod.h5', "r+")

BlockingIOError: [Errno 11] Unable to synchronously open file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')

In [9]:
import h5py
import pyUSID as usid

h5_main = usid.hdf_utils.find_dataset(h5_f, "Raw_Data")

In [20]:
h5_main[0].spec_dim_descriptors
h5_main[0].spec_dim_labels
h5_main[0].spec_dim_sizes

pos_dims = [usid.Dimension(spec_dim_descriptor, spec_dim_labels, spec_dim_size)]

[165, 96, 2, 2]

In [21]:
import pyUSID as usid

# Assuming h5_main is your main dataset from an HDF5 file
# and has already been defined.

# Extract the spec_dim_descriptors, spec_dim_labels, and spec_dim_sizes
spec_dim_descriptors = h5_main[0].spec_dim_descriptors
spec_dim_labels = h5_main[0].spec_dim_labels
spec_dim_sizes = h5_main[0].spec_dim_sizes

# Create the list of usid.Dimension objects
spec_dim = [usid.Dimension(descriptor, label, size)
            for descriptor, label, size in zip(spec_dim_descriptors, spec_dim_labels, spec_dim_sizes)]

In [23]:
# Assuming h5_main is your main dataset from an HDF5 file
# and has already been defined.

# Extract the spec_dim_descriptors, spec_dim_labels, and spec_dim_sizes
pos_dim_descriptors = h5_main[0].pos_dim_descriptors
pos_dim_labels = h5_main[0].pos_dim_labels
pos_dim_sizes = h5_main[0].pos_dim_sizes

# Create the list of usid.Dimension objects
pos_dim = [usid.Dimension(descriptor, label, size)
            for descriptor, label, size in zip(pos_dim_descriptors, pos_dim_labels, pos_dim_sizes)]

In [25]:
pos_dim

[Y (um): generic (Y) mode:DimType.DEFAULT : [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
  36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
  54. 55. 56. 57. 58. 59.],
 X (um): generic (X) mode:DimType.DEFAULT : [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
  36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
  54. 55. 56. 57. 58. 59.]]

In [7]:
import h5py
import pyUSID as usid

with h5py.File('./Data/data_raw_unmod.h5', "r+") as h5_f:
    h5_main = usid.hdf_utils.find_dataset(h5_f, "Raw_Data")[0]
    print(usid.hdf_utils.check_if_main(h5_main))
    print(h5_main)

True
<HDF5 dataset "Raw_Data": shape (3600, 63360), type "<c8">
located at: 
	/Measurement_000/Channel_000/Raw_Data 
Data contains: 
	Piezoresponse (V) 
Data dimensions and original shape: 
Position Dimensions: 
	Y - size: 60 
	X - size: 60 
Spectroscopic Dimensions: 
	Frequency - size: 165 
	DC_Offset - size: 96 
	Field - size: 2 
	Cycle - size: 2
Data Type:
	complex64


In [28]:
def get_original_data(file):
    """
    get_original_data gets the raw BE data as a complex value

    Returns:
        np.array: BE data as a complex number
    """
    with h5py.File(file, "r+") as h5_f:
        return h5_f["Measurement_000"]["Channel_000"]["Raw_Data"][:]

In [31]:
import numpy as np

def generate_noisy_data_records(file,
                                noise_levels,
                                basegroup='Measurement_000',
                                verbose=False,
                                noise_STD=None):

    if verbose:
        print(f"The STD of the data is: {noise_STD}")

    with h5py.File(file, "r+") as h5_f:

        # iterates through the noise levels provided
        for noise_level in noise_levels:

            if verbose:
                print(f"Adding noise level {noise_level}")

            # computes the noise level
            noise_level_ = noise_STD * noise_level

            # computes the real and imaginary components of the noise
            noise_real = np.random.uniform(-1*noise_level_,
                                            noise_level_, (3600, 165*4*96))
            noise_imag = np.random.uniform(-1*noise_level_,
                                           noise_level_, (3600, 165*4*96))

            # adds the noise to the original data
            noise = noise_real+noise_imag*1.0j
            data = get_original_data(file) + noise

            h5_main = usid.hdf_utils.find_dataset(h5_f, "Raw_Data")[0]

            # writes the noise record to the pyUSID file
            usid.hdf_utils.write_main_dataset(h5_f[basegroup],  # parent group
                                                data,  # data to be written
                                                # Name of the main dataset
                                                f'Noisy_Data_{noise_level}',
                                                'Piezoresponse',  # quantity
                                                'V',  # units
                                                pos_dim,  # position dimensions
                                                spec_dim,  # spectroscopic dimensions
                                                h5_pos_inds=h5_main.h5_pos_inds,
                                                h5_pos_vals=h5_main.h5_pos_vals,
                                                h5_spec_inds=h5_main.h5_spec_inds,
                                                h5_spec_vals=h5_main.h5_spec_vals,
                                                dtype=np.complex64,
                                                compression='gzip')

In [32]:
generate_noisy_data_records('./Data/data_raw_unmod.h5', noise_levels=np.arange(1, 9),
                            verbose=True,
                            noise_STD=.1)

The STD of the data is: 0.1
Adding noise level 1
Adding noise level 2
Adding noise level 3
Adding noise level 4
